<a href="https://colab.research.google.com/github/NLP-KazNU/Additional/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22stt_ttt_ipynb_qazaq_bashqurt%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.2 MB/s 
     |████████████████████████████████| 182 kB 77.4 MB/s 
     |████████████████████████████████| 7.6 MB 68.1 MB/s 


In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("sammy786/wav2vec2-xlsr-bashkir")
model = Wav2Vec2ForCTC.from_pretrained("sammy786/wav2vec2-xlsr-bashkir")

Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

In [ ]:
file_path = "bashkir.wav"
speech, rate = librosa.load(file_path,sr=16000)

In [ ]:
%%time
input_values = tokenizer(speech, return_tensors = 'pt').input_values

#Store logits (non-normalized predictions)
logits = model(input_values).logits

#Store predicted id's
predicted_ids = torch.argmax(logits, dim =-1)

#decode the audio to generate text
transcript = tokenizer.decode(predicted_ids[0])

CPU times: user 19.6 s, sys: 49.6 ms, total: 19.6 s
Wall time: 19.9 s


In [ ]:
print(transcript)

беренсе дәрес башҡорт теленең үҙенсәлектәре


In [ ]:
with open("test.txt", "w") as f:
    f.write(transcript)

In [ ]:
import re
import os
import xlrd, xlwt


#------------------------- WORD in SL = STEM + ENDING in SL -------------------------#

def splitting_by_words(text):
    result = re.findall("\w+\'?\\w+", text)
    return result


def sorting_endings(endings_file_name):
    endings_wb = xlrd.open_workbook(endings_file_name)
    endings_sh = endings_wb.sheet_by_index(0)
    endings = []
    for rownum in range(endings_sh.nrows-1):
        ending = endings_sh.cell(rownum+1,2).value
        if '\ufeff' or '-' in ending:
            ending = ending.replace('\ufeff', '')
            ending = ending.replace('-', '')
        endings.append(ending)

    sorted_endings = sorted(endings, key=len, reverse=True)
    ###print(sorted_endings)

    return sorted_endings


def stem(word, endings, stems_file_name):
    stems_wb = xlrd.open_workbook(stems_file_name)
    stems_sh = stems_wb.sheet_by_index(0)
    stems = []
    for rownum in range(stems_sh.nrows-1):
        stem = stems_sh.cell(rownum+1,0).value
        if '\ufeff' in stem:
            stem = stem.replace('\ufeff', '')
        stems.append(stem)

    stems_list = sorted(stems, key=len, reverse=True)
    ###print(stems_list)
    
    word_len = len(word)
    min_len_of_word = 2

    if word_len > min_len_of_word:
        n = word_len - min_len_of_word

        if word in stems_list:
            rez_stem = word
            rez_ending = ""

        else:
            i = n+1
            while i > 0:
                word_ending = word[word_len - (i-1):]
                stem = word[:word_len-len(word_ending)]                         
                for ending in endings:
                    if word_ending == ending:
                        if stem in stems_list:
                            rez_stem = stem
                            rez_ending = word_ending
                            i = 0
                if word_ending == '':
                    if stem in stems_list:
                        rez_stem = stem
                        rez_ending = word_ending
                        i = 0
                    else:
                        j = n+1
                        while j > 0:
                            word_ending = word[word_len - (j-1):]
                            stem = word[:word_len-len(word_ending)]
                            for ending in endings:
                                if word_ending == ending:
                                    rez_stem = stem
                                    rez_ending = word_ending
                                    j = 0
                                    i = 0
                            if word_ending == '':
                                rez_stem = word
                                rez_ending = word_ending
                                j = 0
                                i = 0
                            j = j-1
                i = i-1

    else:
        rez_stem = word
        rez_ending = ""
                        
    return rez_stem, rez_ending
    

def stemming(tfile_name, endings, stopwords_file_name, stems_file_name):
    text_file = open(tfile_name, 'r', encoding="utf-8")
    text_file = text_file.read()

    stopwords_wb = xlrd.open_workbook(stopwords_file_name)
    stopwords_sh = stopwords_wb.sheet_by_index(0)
    stopwords = []
    for rownum in range(stopwords_sh.nrows-1):
        stopword = stopwords_sh.cell(rownum+1,0).value
        if '\ufeff' in stopword:
            stopword = stopword.replace('\ufeff', '')
        stopwords.append(stopword)
    stopwords_list = sorted(stopwords, key=len, reverse=True)
    ###print(stopwords_list)
        
    text = splitting_by_words(text_file)
    ###print("text=",text)
    res_text = []

    rim_cifry = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi', 'xxiv']

    for word in text:
        word = word.lower()
        if word not in res_text:
            if word.isnumeric() or word in rim_cifry:
                continue
            res_text.append(word)

    result_words  = [word for word in res_text if word not in stopwords_list]
    ##print("rez=",result_words)
    
    text_by_stemending = {}
    for word in result_words:
        stemm, endingg = stem(word, endings, stems_file_name)
        text_by_stemending.update({word: [stemm, endingg]})
    ##print(text_by_stemending)
       
    return text_by_stemending


#------------------------- STEM + ENDING in SL = STEM + ENDING in TL = WORD in TL -------------------------#

def match_by_table(endings_file, stems_file, stemsandendings_insl):
    stems_wb = xlrd.open_workbook(stems_file)
    stems_sh = stems_wb.sheet_by_index(0)
    stems = {}
    for rownum in range(stems_sh.nrows-1):
        stem_in_sl = stems_sh.cell(rownum+1,1).value
        stem_in_tl = stems_sh.cell(rownum+1,0).value
        stems.update({stem_in_sl: stem_in_tl})
    ###print("Stems ", stems)

    endings_wb = xlrd.open_workbook(endings_file)
    endings_sh = endings_wb.sheet_by_index(0)
    endings = {}
    for rownum in range(endings_sh.nrows-1):
        ending_in_sl = endings_sh.cell(rownum+1,1).value
        ending_in_sl = ending_in_sl.replace('-', '')
        ending_in_tl = endings_sh.cell(rownum+1,4).value
        endings.update({ending_in_sl: ending_in_tl})
    ###print("Endings ", endings)

    matched_words = {}
    for word_in_sl in stemsandendings_insl.keys():
        ###print("*****", word_in_sl, "*****")
        stem_in_sl = stemsandendings_insl[word_in_sl][0]
        ending_in_sl = stemsandendings_insl[word_in_sl][1]
        #print(stem_in_sl, ending_in_sl)
        if stem_in_sl in stems.keys():
            stem_in_tl = stems[stem_in_sl]
            ###print(stem_in_tl)
        else:
            stem_in_tl = stem_in_sl
            ###print(stem_in_tl)
        if ending_in_sl in endings.keys():
            ending_in_tl = endings[ending_in_sl]
            ###print(ending_in_tl)
        else:
            ending_in_tl = ending_in_sl
            ###print(ending_in_tl)
        word_in_tl = stem_in_tl + ending_in_tl
        matched_words.update({word_in_sl: word_in_tl})
        #print(word_in_sl, word_in_tl)

    return matched_words

def replace_sltotl(text_file, matchings, stopwords_file_name):
    stopwords_wb = xlrd.open_workbook(stopwords_file_name)
    stopwords_sh = stopwords_wb.sheet_by_index(0)
    stopwords = {}
    for rownum in range(stopwords_sh.nrows-1):
        stopword_in_sl = stopwords_sh.cell(rownum+1,0).value
        stopword_in_tl = stopwords_sh.cell(rownum+1,1).value
        stopwords.update({stopword_in_sl: stopword_in_tl})
    ###print(stopwords)

    text_file = open(text_file, 'r', encoding="utf-8")
    text_file = text_file.read()
    punctuations = ['.', ',', '?', '!', ':', ';', '(', ')', '"', '«', '»']
    for punctn in punctuations:
        text_file = text_file.replace(punctn, " "+punctn+" ")
    text_file = text_file.replace("  ", " ")
    print("TEXT in SL")
    # print(text_file)
    
    for word in matchings:
        ###print(word)
        if f"{word} " in text_file.lower():
            ###print(matchings[word])
            text_file = text_file.lower().replace(f"{word} ", matchings[word]+" ")

    for stopword in stopwords:
        ###print(stopword)
        if f" {stopword} " in text_file.lower():
            ###print(stopwords[stopword])
            text_file = text_file.lower().replace(f" {stopword} ", " "+stopwords[stopword]+" ")
            
    print("\nTEXT in TL")
    translated_text = text_file
    
    return translated_text

stopwords_file_name = "qaz-bash-stopwords.xlsx"

endings_file_name = "qaz-bash-tab.xlsx"
endings = sorting_endings(endings_file_name)
###print(endings)

text_file_name = "test.txt"
stems_file_name = "qaz-bash-stems.xlsx"
text_by_stemending = stemming(text_file_name, endings, stopwords_file_name, stems_file_name)
###print(text_by_stemending)

matchings = match_by_table(endings_file_name, stems_file_name, text_by_stemending)
###print(matchings)

translated_text = replace_sltotl(text_file_name, matchings, stopwords_file_name)
print(translated_text)

TEXT in SL

TEXT in TL
бірінші сабақ башқұрт тілінің үҙенсәлектәре


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!huggingface-cli login

/bin/bash: huggingface-cli: command not found


In [ ]:
!wget https://issai.nu.edu.kz/wp-content/uploads/2022/03/parallelwavegan_male2_checkpoint.zip

--2022-12-07 14:36:42--  https://issai.nu.edu.kz/wp-content/uploads/2022/03/parallelwavegan_male2_checkpoint.zip
Resolving issai.nu.edu.kz (issai.nu.edu.kz)... 178.91.253.144
Connecting to issai.nu.edu.kz (issai.nu.edu.kz)|178.91.253.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15610294 (15M) [application/zip]
Saving to: ‘parallelwavegan_male2_checkpoint.zip’

parallelwavegan_mal 100%[===================>]  14.89M  14.5MB/s    in 1.0s    

2022-12-07 14:36:43 (14.5 MB/s) - ‘parallelwavegan_male2_checkpoint.zip’ saved [15610294/15610294]



In [ ]:
!unzip parallelwavegan_male2_checkpoint.zip

Archive:  parallelwavegan_male2_checkpoint.zip
   creating: parallelwavegan_male2_checkpoint/
  inflating: parallelwavegan_male2_checkpoint/config.yml  
  inflating: parallelwavegan_male2_checkpoint/checkpoint-400000steps.pkl  


In [ ]:
!wget https://issai.nu.edu.kz/wp-content/uploads/2022/03/kaztts_male2_tacotron2_train.loss.ave.zip

--2022-12-07 14:31:04--  https://issai.nu.edu.kz/wp-content/uploads/2022/03/kaztts_male2_tacotron2_train.loss.ave.zip
Resolving issai.nu.edu.kz (issai.nu.edu.kz)... 178.91.253.144
Connecting to issai.nu.edu.kz (issai.nu.edu.kz)|178.91.253.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107280461 (102M) [application/zip]
Saving to: ‘kaztts_male2_tacotron2_train.loss.ave.zip’

kaztts_male2_tacotr 100%[===================>] 102.31M  31.6MB/s    in 3.7s    

2022-12-07 14:31:08 (27.3 MB/s) - ‘kaztts_male2_tacotron2_train.loss.ave.zip’ saved [107280461/107280461]



In [ ]:
!unzip kaztts_male2_tacotron2_train.loss.ave.zip

Archive:  kaztts_male2_tacotron2_train.loss.ave.zip
 extracting: meta.yaml               
 extracting: exp/tts_train_raw_char/config.yaml  
 extracting: exp/tts_train_raw_char/train.loss.ave_5best.pth  
 extracting: exp/tts_train_raw_char/images/gpu_max_cached_mem_GB.png  
 extracting: exp/tts_train_raw_char/images/iter_time.png  
 extracting: exp/tts_train_raw_char/images/optim0_lr0.png  
 extracting: exp/tts_train_raw_char/images/train_time.png  
 extracting: exp/tts_train_raw_char/images/mse_loss.png  
 extracting: exp/tts_train_raw_char/images/forward_time.png  
 extracting: exp/tts_train_raw_char/images/backward_time.png  
 extracting: exp/tts_train_raw_char/images/loss.png  
 extracting: exp/tts_train_raw_char/images/l1_loss.png  
 extracting: exp/tts_train_raw_char/images/optim_step_time.png  
 extracting: exp/tts_train_raw_char/images/attn_loss.png  
 extracting: exp/tts_train_raw_char/images/bce_loss.png  
 extracting: exp/tts_stats_raw_char/train/feats_stats.npz  


In [ ]:
!pip install parallel_wavegan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install espnet[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


'''
2022.06.03
kazakh_to_ipa() <> ipa_to_kazakh()
test_kazakh()
turkish_to_ipa() <> ipa_to_turkish()
test_turkish()

2022.07.05
kyrgyz_to_ipa() <> ipa_to_kyrgyz()
test_kyrgyz()
uzbek_to_ipa() <> ipa_to_uzbek()
test_uzbek()
azerbaijani_to_ipa() <> ipa_to_azerbaijani()
test_azerbaijani()
turkmen_to_ipa() <> ipa_to_turkmen()
test_turkmen()

2022.07.07
tatar_to_ipa() <> ipa_to_tatar()
test_tatar()
bashkir_to_ipa() <> ipa_to_bashkir()
test_bashkir()
sakha_to_ipa() <> ipa_to_sakha()
test_sakha()

2022.07.12
experimentally added î and â to turkish_to_ipa()

2022.08.04
uyghur_to_ipa() <> ipa_to_uyghur()
'''

import re
    

# kazakh scripts

def kazakh_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list.
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # three-sound convenience vowels:
    text = re.sub("[Юю]", "ǔ", text)

    # two-sound convenience consonants:
    text = re.sub("[Цц]", "š", text)
    text = re.sub("[Чч]", "ʆ", text)

    # two-sound convenience vowels:
    text = re.sub("[Яя]", "ǎ", text)
    text = re.sub("[Ее]", "ě", text)
    text = re.sub("[Ёё]", "ǒ", text)
    text = re.sub("[Ии]", "ǐ", text)
    text = re.sub("[Уу]", "u", text)

    # single-sound consonants:
    text = re.sub("[Бб]", "b", text)
    text = re.sub("[Вв]", "v", text)
    text = re.sub("[Гг]", "g", text)
    text = re.sub("[Ғғ]", "ɣ", text)
    text = re.sub("[Дд]", "d", text)
    text = re.sub("[Жж]", "ʒ", text)
    text = re.sub("[Зз]", "z", text)
    text = re.sub("[Йй]", "j", text)
    text = re.sub("[Кк]", "k", text)
    text = re.sub("[Ққ]", "q", text)
    text = re.sub("[Лл]", "l", text)
    text = re.sub("[Мм]", "m", text)
    text = re.sub("[Нн]", "n", text)
    text = re.sub("[Ңң]", "ŋ", text)
    text = re.sub("[Пп]", "p", text)
    text = re.sub("[Рр]", "r", text)
    text = re.sub("[Сс]", "s", text)
    text = re.sub("[Тт]", "t", text)
    text = re.sub("[Фф]", "f", text)
    text = re.sub("[Хх]", "x", text)
    text = re.sub("[Һһ]", "h", text)
    text = re.sub("[Шш]", "ʃ", text)
    text = re.sub("[Щщ]", "ɕ", text)
    text = re.sub("[Ъъ]", "ʔ", text)
    text = re.sub("[Ьь]", "ʲ", text)

    # single-sound vowels:
    text = re.sub("[Аа]", "ɑ", text)
    text = re.sub("[Әә]", "æ", text)
    text = re.sub("[Оо]", "ɔ", text)
    text = re.sub("[Өө]", "ɵ", text)
    text = re.sub("[Ұұ]", "ʊ", text)
    text = re.sub("[Үү]", "ʏ", text)
    text = re.sub("[Ыы]", "ɤ", text)
    text = re.sub("[Іі]", "ɪ", text)
    text = re.sub("[Ээ]", "e", text)

    # rules

    '''
    rule 1:
    if [æ], [ě], [ɵ], [ʏ], [ɪ] are followed by [l] and [l] is NOT followed by [æ], [ě], [ɵ], [ʏ], [ɪ], or [ʲ],
    use [ł] instead of [l] (e.g., [kěł], [kěłdɪ], but [kělěmɪn], [marsělʲ]).
    '''
    text = re.sub(r"([æěɵʏɪ])(l)([^æěɵʏɪʲ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letters "о" and "ө", [ɔ] and [ɵ] at the beginning of a word are preceded by [w] (e.g., осы [wɔsɤ] not [ɔsɤ], өзі [wɵzɪ] not [ɵzɪ]).
    '''
    text = re.sub(r"\b([ɔɵ])", r"w\1", text)

    '''
    rule 3
    if the letter "у" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆʲ])", r"w\1", text)

    '''
    rule 4:
    if the letter "и" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆʲ])", r"i\1", text)

    return text

def ipa_to_kazakh(text):
    # three-sound convenience vowels:
    text = re.sub("ǔ", "ю", text)

    # two-sound convenience consonants:
    text = re.sub("š", "ц", text)
    text = re.sub("ʆ", "ч", text)

    # two-sound convenience vowels:
    text = re.sub("ǎ", "я", text)
    text = re.sub("ě", "е", text)
    text = re.sub("ǒ", "ё", text)
    text = re.sub("ǐ", "и", text)
    text = re.sub("u", "у", text)

    # single-sound consonants:
    text = re.sub("b", "б", text)
    text = re.sub("v", "в", text)
    text = re.sub("g", "г", text)
    text = re.sub("ɣ", "ғ",  text)
    text = re.sub("d", "д", text)
    text = re.sub("ʒ", "ж", text)
    text = re.sub("z", "з", text)
    text = re.sub("j", "й", text)
    text = re.sub("k", "к", text)
    text = re.sub("q", "қ", text)
    text = re.sub("l", "л", text)
    text = re.sub("m", "м", text)
    text = re.sub("n", "н", text)
    text = re.sub("ŋ", "ң", text)
    text = re.sub("p", "п", text)
    text = re.sub("r", "р", text)
    text = re.sub("s", "с", text)
    text = re.sub("t", "т", text)
    text = re.sub("f", "ф", text)
    text = re.sub("x", "х", text)
    text = re.sub("h", "һ", text)
    text = re.sub("ʃ", "ш", text)
    text = re.sub("ɕ", "щ", text)
    text = re.sub("ʔ", "ъ", text)
    text = re.sub("ʲ", "ь", text)

    # single-sound vowels:
    text = re.sub("ɑ", "а", text)
    text = re.sub("æ", "ә", text)
    text = re.sub("ɔ", "о", text)
    text = re.sub("ɵ", "ө", text)
    text = re.sub("ʊ", "ұ", text)
    text = re.sub("ʏ", "ү", text)
    text = re.sub("ɤ", "ы", text)
    text = re.sub("ɪ", "і", text)
    text = re.sub("e", "э", text)

    # anti-rules

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([әеөүі])(ł)([^әеөүіь])", r"\1л\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"\bw([оө])", r"\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"w([бвгғджзйкқлмнңпрстфхһцчшщъьчц])", r"у\1", text)

    '''
    anti-rule 4:
    the symbol [i] is used in one case only, so we can just replace it for и.
    '''
    text = re.sub(r"i", r"и", text)

    '''
    anti-rules for Turkish and Kyrgyz Ǯ, Turkish ł, azerbaijani ḡ, sakha ɲ
    '''
    text = re.sub(r"w([Ǯ])", r"у\1", text)
    text = re.sub(r"Ǯ", r"дж", text)
    text = re.sub(r"ł", r"ль", text)
    text = re.sub(r"ḡ", r"гь", text)
    text = re.sub(r"ɲ", r"нь", text)

    return text

# testing kazakh scripts

def test_kazakh(text):
    input_text = text.lower().split()
    output_text = ipa_to_kazakh(kazakh_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# turkish scripts

def turkish_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list.
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # two-sound convenience consonants:
    text = re.sub("[Cc]", "Ǯ", text)
    text = re.sub("[Çç]", "ʆ", text)

    # two-sound convenience vowels:
    text = re.sub("[İi]", "ǐ", text)
    text = re.sub("[Uu]", "u", text)

    # single-sound consonants:
    text = re.sub("[Jj]", "ʒ", text)
    text = re.sub("[Yy]", "j", text)
    text = re.sub("[Bb]", "b", text)
    text = re.sub("[Dd]", "d", text)
    text = re.sub("[Ff]", "f", text)
    text = re.sub("[Gg]", "g", text)
    text = re.sub("[Ğğ]", "ɣ", text)
    text = re.sub("[Hh]", "h", text)
    text = re.sub("[Kk]", "k", text)
    text = re.sub("[Ll]", "l", text)
    text = re.sub("[Mm]", "m", text)
    text = re.sub("[Nn]", "n", text)
    text = re.sub("[Pp]", "p", text)
    text = re.sub("[Rr]", "r", text)
    text = re.sub("[Ss]", "s", text)
    text = re.sub("[Şş]", "ʃ", text)
    text = re.sub("[Tt]", "t", text)
    text = re.sub("[Vv]", "v", text)
    text = re.sub("[Zz]", "z", text)

    # single-sound vowels:
    text = re.sub("[Aa]", "ɑ", text)
    text = re.sub("[Ee]", "e", text)
    text = re.sub("[Iı]", "ɤ", text)
    text = re.sub("[Oo]", "ɔ", text)
    text = re.sub("[Öö]", "ɵ", text)
    text = re.sub("[Üü]", "ʏ", text)
    text = re.sub("[Îî]", "ǐ", text) # experimentally added
    text = re.sub("[Ââ]", "ɑ", text) # experimentally added


    '''
    rule 1:
    if [e], [ɵ], [ʏ], [i] are followed by [l] and [l] is NOT followed by [e], [ɵ], [ʏ], or [i],
    use [ł] instead of [l] (e.g., [geł], [gełdi], but [gelecek]).
    '''
    text = re.sub(r"([eɵʏǐ])(l)([^eɵʏǐ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letter "u" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdʒzklłmnprstfhʃʆǮ])", r"w\1", text)

    '''
    rule 3:
    if the letter "i" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdʒzklłmnprstfhʃʆǮ])", r"i\1", text)

    return text

def ipa_to_turkish(text):
    # two-sound convenience consonants:
    text = re.sub("Ǯ", "c", text)
    text = re.sub("ʆ", "ç", text)

    # single-sound consonants:
    text = re.sub("j", "y", text)
    text = re.sub("ʒ", "j", text)
    text = re.sub("b", "b", text)
    text = re.sub("d", "d", text)
    text = re.sub("f", "f", text)
    text = re.sub("g", "g", text)
    text = re.sub("ɣ", "ğ", text)
    text = re.sub("h", "h", text)
    text = re.sub("k", "k", text)
    text = re.sub("l", "l", text)
    text = re.sub("m", "m", text)
    text = re.sub("n", "n", text)
    text = re.sub("p", "p", text)
    text = re.sub("r", "r", text)
    text = re.sub("s", "s", text)
    text = re.sub("ʃ", "ş", text)
    text = re.sub("t", "t", text)
    text = re.sub("v", "v", text)
    text = re.sub("z", "z", text)

    # single-sound vowels:
    text = re.sub("ɑ", "a", text)
    text = re.sub("e", "e", text)
    text = re.sub("ɤ", "ı", text)
    text = re.sub("ǐ", "i", text)
    text = re.sub("ɔ", "o", text)
    text = re.sub("ɵ", "ö", text)
    text = re.sub("ʏ", "ü", text)

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([eöüi])(ł)([^eöüi])", r"\1l\3", text)

    '''
    anti-rule 2:
    the symbol [w] is used in one case only, so we can just replace it for u.
    '''
    text = re.sub(r"w", r"u", text)

    '''
    anti-rule 3:
    the symbol [i] is used in one case only, so we can just replace it for i.
    '''
    text = re.sub(r"i", r"i", text)

    return text

# testing turkish scripts

def test_turkish(text):
    input_text = text.lower().split()
    output_text = ipa_to_turkish(turkish_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# kyrgyz scripts

def kyrgyz_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # three-sound convenience vowels:
    text = re.sub("[Юю]", "ǔ", text)

    # two-sound convenience consonants:
    text = re.sub("[Цц]", "š", text)
    text = re.sub("[Чч]", "ʆ", text)
    text = re.sub("[Жж]", "Ǯ", text)

    # two-sound convenience vowels:
    text = re.sub("[Яя]", "ǎ", text)
    text = re.sub("[Ее]", "ě", text)
    text = re.sub("[Ёё]", "ǒ", text)
    text = re.sub("[Ии]", "ǐ", text)
    text = re.sub("[Уу]", "u", text)

    # single-sound consonants:
    text = re.sub("[Бб]", "b", text)
    text = re.sub("[Вв]", "v", text)
    text = re.sub("[Гг]", "g", text)
    text = re.sub("[Дд]", "d", text)
    text = re.sub("[Зз]", "z", text)
    text = re.sub("[Йй]", "j", text)
    text = re.sub("[Кк]", "k", text)
    text = re.sub("[Лл]", "l", text)
    text = re.sub("[Мм]", "m", text)
    text = re.sub("[Нн]", "n", text)
    text = re.sub("[Ңң]", "ŋ", text)
    text = re.sub("[Пп]", "p", text)
    text = re.sub("[Рр]", "r", text)
    text = re.sub("[Сс]", "s", text)
    text = re.sub("[Тт]", "t", text)
    text = re.sub("[Фф]", "f", text)
    text = re.sub("[Хх]", "x", text)
    text = re.sub("[Шш]", "ʃ", text)
    text = re.sub("[Щщ]", "ɕ", text)
    text = re.sub("[Ъъ]", "ʔ", text)
    text = re.sub("[Ьь]", "ʲ", text)

    # single-sound vowels:
    text = re.sub("[Аа]", "ɑ", text)
    text = re.sub("[Оо]", "ɔ", text)
    text = re.sub("[Өө]", "ɵ", text)
    text = re.sub("[Үү]", "ʏ", text)
    text = re.sub("[Ыы]", "ɤ", text)
    text = re.sub("[Ээ]", "e", text)

    # rules 1-4 are similar to those for Kazakh:

    '''
    rule 1:
    if [ě], [ɵ], [ʏ], are followed by [l] and [l] is NOT followed by [ě], [ɵ], [ʏ], or [ʲ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([ɵʏě])(l)([^ɵʏěʲ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letters "о" and "ө", [ɔ] and [ɵ] at the beginning of a word are followed by [w] (e.g., осы [wɔsɤ] not [ɔsɤ], өзі [wɵzɪ] not [ɵzɪ]).
    '''
    text = re.sub(r"\b([ɔɵ])", r"w\1", text)

    '''
    rule 3
    if the letter "у" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdzjkqlłmnŋprstfxhʃɕʔšʆǮʲ])", r"w\1", text)

    '''
    rule 4:
    if the letter "и" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdzjkqlłmnŋprstfxhʃɕʔšʆǮʲ])", r"i\1", text)

    # rules 5-6 are specific to Kyrgyz:

    '''
    rule 5
    ɑ|ɔ|u|ɤ + k + ɑ|ɔ|u|ɤ
    '''
    text = re.sub(r"([ɑɔwɤ])k", r"\1q", text)
    text = re.sub(r"k([ɑɔuɤ])", r"q\1", text)

    '''
    rule 6
    ɑ|ɔ|u|ɤ + g + ɑ|ɔ|u|ɤ
    '''
    text = re.sub(r"([ɑɔwɤ])g", r"\1ɣ", text)
    text = re.sub(r"g([ɑɔuɤ])", r"ɣ\1", text)

    return text

def ipa_to_kyrgyz(text):
    # three-sound convenience vowels:
    text = re.sub("ǔ", "ю", text)

    # two-sound convenience consonants:
    text = re.sub("š", "ц", text)
    text = re.sub("ʆ", "ч", text)
    text = re.sub("Ǯ", "ж", text)

    # two-sound convenience vowels:
    text = re.sub("ǎ", "я", text)
    text = re.sub("ě", "е", text)
    text = re.sub("ǒ", "ё", text)
    text = re.sub("ǐ", "и", text)
    text = re.sub("u", "у", text)

    # single-sound consonants:
    text = re.sub("b", "б", text)
    text = re.sub("v", "в", text)
    text = re.sub("g", "г", text)
    text = re.sub("ɣ", "г", text)
    text = re.sub("d", "д", text)
    text = re.sub("z", "з", text)
    text = re.sub("j", "й", text)
    text = re.sub("k", "к", text)
    text = re.sub("l", "л", text)
    text = re.sub("m", "м", text)
    text = re.sub("n", "н", text)
    text = re.sub("ŋ", "ң", text)
    text = re.sub("p", "п", text)
    text = re.sub("q", "к", text)
    text = re.sub("r", "р", text)
    text = re.sub("s", "с", text)
    text = re.sub("t", "т", text)
    text = re.sub("f", "ф", text)
    text = re.sub("x", "х", text)
    text = re.sub("ʃ", "ш", text)
    text = re.sub("ɕ", "щ", text)
    text = re.sub("ʔ", "ъ", text)
    text = re.sub("ʲ", "ь", text)

    # single-sound vowels:
    text = re.sub("ɑ", "а", text)
    text = re.sub("ɔ", "о", text)
    text = re.sub("ɵ", "ө", text)
    text = re.sub("ʏ", "ү", text)
    text = re.sub("ɤ", "ы", text)
    text = re.sub("e", "э", text)

    # anti-rules 1-4 are similar to those for Kazakh:

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([өүе])(ł)([^өүеʲ])", r"\1л\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"\bw([оө])", r"\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"w([бвгдзйклмнңпрстфхцчшщъьчцж])", r"у\1", text)

    '''
    anti-rule 4:
    '''
    text = re.sub(r"i([бвгдзйклмнңпрстфхцчшщъьчцж])", r"и\1", text)

    return text

# testing kyrgyz scripts

def test_kyrgyz(text):
    input_text = text.lower().split()
    output_text = ipa_to_kyrgyz(kyrgyz_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# uzbek scripts

def uzbek_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # two-sound convenience consonants:
    text = re.sub("[Jj]", "Ǯ", text)
    text = re.sub("Ch", "ʆ", text)
    text = re.sub("ch", "ʆ", text)

    # two-sound convenience vowels:
    text = re.sub("[Ii]", "ǐ", text)
    text = re.sub("[Uu]", "u", text)

    # single-sound consonants:
    text = re.sub("[Bb]", "b", text)
    text = re.sub("[Dd]", "d", text)
    text = re.sub("[Ff]", "f", text)
    text = re.sub("G‘", "ɣ", text)
    text = re.sub("g‘", "ɣ", text)
    text = re.sub("[Gg]", "g", text)
    text = re.sub("[Hh]", "h", text)
    text = re.sub("[Kk]", "k", text)
    text = re.sub("[Ll]", "l", text)
    text = re.sub("[Mm]", "m", text)
    text = re.sub("[Nn]", "n", text)
    text = re.sub("Ng", "ŋ", text)
    text = re.sub("ng", "ŋ", text)
    text = re.sub("[Pp]", "p", text)
    text = re.sub("[Qq]", "q", text)
    text = re.sub("[Rr]", "r", text)
    text = re.sub("[Ss]", "s", text)
    text = re.sub("Sh", "ʃ", text)
    text = re.sub("sh", "ʃ", text)
    text = re.sub("[Tt]", "t", text)
    text = re.sub("[Vv]", "v", text)
    text = re.sub("[Xx]", "x", text)
    text = re.sub("[Yy]", "j", text)
    text = re.sub("[Zz]", "z", text)

    # single-sound vowels:
    text = re.sub("[Aa]", "æ", text)
    text = re.sub("[Ee]", "e", text)
    text = re.sub("Oʻ", "ɵ", text)
    text = re.sub("oʻ", "ɵ", text)
    text = re.sub("[Oo]", "ɔ", text)

    # hard sign
    text = re.sub("'", "ʔ", text)

    '''
    rule 1:
    if [æ], [e], [ɵ], [ǐ] are followed by [l] and [l] is NOT followed by [æ], [e], [ɵ], [ǐ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æɵǐe])(l)([^æɵǐe])", r"\1ł\3", text)

    '''
    rule 2:
    if the letter "u" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdzjkqlłmnŋprstfxhʃʔʆǮ])", r"w\1", text)

    '''
    rule 3:
    if the letter "i" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdzjkqlłmnŋprstfxhʃʔʆǮ])", r"i\1", text)

    return text

def ipa_to_uzbek(text):
    # two-sound convenience consonants:
    text = re.sub("j", "y", text)  # exception! precedence issue
    text = re.sub("Ǯ", "j", text)
    text = re.sub("ʆ", "ch", text)

    # two-sound convenience vowels:
    text = re.sub("ǐ", "i", text)
    text = re.sub("u", "u", text)

    # single-sound convenience consonants:
    text = re.sub("b", "b", text)
    text = re.sub("d", "d", text)
    text = re.sub("f", "f", text)
    text = re.sub("g", "g", text)
    text = re.sub("ɣ", "g‘", text)
    text = re.sub("h", "h", text)
    text = re.sub("k", "k", text)
    text = re.sub("l", "l", text)
    text = re.sub("m", "m", text)
    text = re.sub("n", "n", text)
    text = re.sub("ŋ", "ng", text)
    text = re.sub("p", "p", text)
    text = re.sub("q", "q", text)
    text = re.sub("r", "r", text)
    text = re.sub("s", "s", text)
    text = re.sub("ʃ", "sh", text)
    text = re.sub("t", "t", text)
    text = re.sub("v", "v", text)
    text = re.sub("x", "x", text)
    text = re.sub("z", "z", text)

    # single-sound convenience vowels:
    text = re.sub("æ", "a", text)
    text = re.sub("e", "e", text)
    text = re.sub("ɵ", "o‘", text)
    text = re.sub("ɔ", "o", text)

    # hard sign
    text = re.sub("ʔ", "'", text)

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([aei‘])(ł)([^aei‘])", r"\1l\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"w([bcvgɣdjzklmnpqrstfhyx])", r"u\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"i([bcvgɣdjzklmnpqrstfhyx])", r"i\1", text)

    return text

# testing uzbek scripts

def test_uzbek(text):
    input_text = text.lower().split()
    output_text = ipa_to_uzbek(uzbek_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# azerbaijani scripts

def azerbaijani_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # two-sound convenience consonants:
    text = re.sub("[Cc]", "Ǯ", text)
    text = re.sub("[Çç]", "ʆ", text)
    text = re.sub("[Gg]", "ḡ", text)

    # two-sound convenience vowels:
    text = re.sub("[İi]", "ǐ", text)
    text = re.sub("[Uu]", "u", text)

    # single-sound consonants:
    text = re.sub("[Jj]", "ʒ", text)
    text = re.sub("[Yy]", "j", text)
    text = re.sub("[Bb]", "b", text)
    text = re.sub("[Dd]", "d", text)
    text = re.sub("[Ff]", "f", text)
    text = re.sub("[Ğğ]", "ɣ", text)
    text = re.sub("[Hh]", "h", text)
    text = re.sub("[Xx]", "x", text)
    text = re.sub("[Kk]", "k", text)
    text = re.sub("[Qq]", "g", text)
    text = re.sub("[Ll]", "l", text)
    text = re.sub("[Mm]", "m", text)
    text = re.sub("[Nn]", "n", text)
    text = re.sub("[Pp]", "p", text)
    text = re.sub("[Rr]", "r", text)
    text = re.sub("[Ss]", "s", text)
    text = re.sub("[Şş]", "ʃ", text)
    text = re.sub("[Tt]", "t", text)
    text = re.sub("[Vv]", "v", text)
    text = re.sub("[Zz]", "z", text)

    # single-sound vowels:
    text = re.sub("[Aa]", "ɑ", text)
    text = re.sub("[Ee]", "e", text)
    text = re.sub("[Əə]", "æ", text)
    text = re.sub("[Iı]", "ɤ", text)
    text = re.sub("[Oo]", "ɔ", text)
    text = re.sub("[Öö]", "ɵ", text)
    text = re.sub("[Üü]", "ʏ", text)

    '''
    rule 1:
    if [æ], [e], [ɵ], [ʏ], [i] are followed by [l] and [l] is NOT followed by [æ], [e], [ɵ], [ʏ], or [i],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æeɵʏǐ])(l)([^æeɵʏǐ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letter "u" [ʊw] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgḡɣdʒzklłmnprstfhxʃʆǮ])", r"w\1", text)

    '''
    rule 3:
    if the letter "i" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgḡɣdʒzklłmnprstfhxʃʆǮ])", r"i\1", text)

    return text

def ipa_to_azerbaijani(text):
    # two-sound convenience consonants:
    text = re.sub("Ǯ", "c", text)
    text = re.sub("ʆ", "ç", text)
    text = re.sub("g", "q", text)  # precedence issue
    text = re.sub("ḡ", "g", text)

    # single-sound consonants:
    text = re.sub("j", "y", text)
    text = re.sub("ʒ", "j", text)
    text = re.sub("b", "b", text)
    text = re.sub("d", "d", text)
    text = re.sub("f", "f", text)
    text = re.sub("ɣ", "ğ", text)
    text = re.sub("h", "h", text)
    text = re.sub("x", "x", text)
    text = re.sub("k", "k", text)
    text = re.sub("l", "l", text)
    text = re.sub("m", "m", text)
    text = re.sub("n", "n", text)
    text = re.sub("p", "p", text)
    text = re.sub("r", "r", text)
    text = re.sub("s", "s", text)
    text = re.sub("ʃ", "ş", text)
    text = re.sub("t", "t", text)
    text = re.sub("v", "v", text)
    text = re.sub("z", "z", text)

    # single-sound vowels:
    text = re.sub("ɑ", "a", text)
    text = re.sub("e", "e", text)
    text = re.sub("æ", "ə", text)
    text = re.sub("ɤ", "ı", text)
    text = re.sub("ǐ", "i", text)
    text = re.sub("ɔ", "o", text)
    text = re.sub("ɵ", "ö", text)
    text = re.sub("ʏ", "ü", text)

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([əeöüiě])(ł)([^əeöüiě])", r"\1l\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"w([bvgğdjzkqlmnprstfhxşçc])", r"u\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"i([bcvgğdjzkqlmnprstfhxşç])", r"i\1", text)

    return text

# testing azerbaijani scripts

def test_azerbaijani(text):
    input_text = text.lower().split()
    output_text = ipa_to_azerbaijani(azerbaijani_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# turkmen scripts

def turkmen_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # two-sound convenience consonants:
    text = re.sub("[Çç]", "ʆ", text)
    text = re.sub("[Jj]", "Ǯ", text)

    # two-sound convenience vowels:
    text = re.sub("[İi]", "ǐ", text)
    text = re.sub("[Uu]", "u", text)

    # single-sound consonants:
    text = re.sub("[Bb]", "b", text)
    text = re.sub("[Dd]", "d", text)
    text = re.sub("[Ff]", "f", text)
    text = re.sub("[Gg]", "g", text)
    text = re.sub("[Hh]", "h", text)
    text = re.sub("[Žž]", "ʒ", text)
    text = re.sub("[Kk]", "k", text)
    text = re.sub("[Ll]", "l", text)
    text = re.sub("[Mm]", "m", text)
    text = re.sub("[Nn]", "n", text)
    text = re.sub("[Ňň]", "ŋ", text)
    text = re.sub("[Pp]", "p", text)
    text = re.sub("[Rr]", "r", text)
    text = re.sub("[Ss]", "s", text)  # θ
    text = re.sub("[Şş]", "ʃ", text)
    text = re.sub("[Tt]", "t", text)
    text = re.sub("[Ww]", "v", text)
    text = re.sub("[Ýý]", "j", text)
    text = re.sub("[Zz]", "z", text)  # ð

    # single-sound vowels:
    text = re.sub("[Aa]", "ɑ", text)
    text = re.sub("[Ää]", "æ", text)
    text = re.sub("[Ee]", "e", text)
    text = re.sub("[Oo]", "ɔ", text)
    text = re.sub("[Öö]", "ɵ", text)
    text = re.sub("[Üü]", "ʏ", text)
    text = re.sub("[Yy]", "ɤ", text)

    # rules:

    '''
    rule 1:
    if [æ], [e], [ɵ], [ʏ], [i] are followed by [l] and [l] is NOT followed by [æ], [e], [ɵ], [ʏ], or [i],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æeɵʏǐ])(l)([^æeɵʏǐ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letter "u" [ʊw] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣqdʒzkqlłmnprstfhʃʆǮw])", r"w\1", text)

    '''
    rule 3:
    if the letter "i" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣqdʒzkqlłmnprstfhʃʆǮ])", r"i\1", text)

    # rules 4-5 are specific to Turkmen:

    '''
    rule 4:
    a, o, u, y + k + a, o, u, y:
    '''
    text = re.sub(r"k([ɑɔuɤ])", r"q\1", text)
    text = re.sub(r"([ɑɔwɤ])k", r"\1q", text)

    '''
    rule 5:
    a, o, u, y + g + a, o, u, y:
    '''
    text = re.sub(r"g([ɑɔuɤ])", r"ɣ\1", text)
    text = re.sub(r"([ɑɔwɤ])g", r"\1ɣ", text)

    return text

def ipa_to_turkmen(text):
    # two-sound convenience consonants:
    text = re.sub("j", "ý", text)  # precedence issue
    text = re.sub("Ǯ", "j", text)
    text = re.sub("ʆ", "ç", text)

    # single-sound consonants: # w --> v can be found where the letter u anti-rule is
    text = re.sub("b", "b", text)
    text = re.sub("d", "d", text)
    text = re.sub("f", "f", text)
    text = re.sub("g", "g", text)
    text = re.sub("ɣ", "g", text)
    text = re.sub("h", "h", text)
    text = re.sub("ʒ", "ž", text)
    text = re.sub("k", "k", text)
    text = re.sub("q", "k", text)
    text = re.sub("l", "l", text)
    text = re.sub("m", "m", text)
    text = re.sub("n", "n", text)
    text = re.sub("ŋ", "ň", text)
    text = re.sub("p", "p", text)
    text = re.sub("r", "r", text)
    text = re.sub("s", "s", text)
    text = re.sub("ʃ", "ş", text)
    text = re.sub("t", "t", text)
    text = re.sub("z", "z", text)

    # single-sound vowels:
    text = re.sub("ɑ", "a", text)
    text = re.sub("e", "e", text)
    text = re.sub("æ", "ä", text)
    text = re.sub("ǐ", "i", text)
    text = re.sub("ɔ", "o", text)
    text = re.sub("ɵ", "ö", text)
    text = re.sub("ʏ", "ü", text)
    text = re.sub("ɤ", "y", text)

    # anti-rules:

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([äeöüiě])(ł)([^äeöüiě])", r"\1l\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"w([bdfghžklmnňprsştýzjçɣqv])", r"u\1", text)  # precedence issue
    text = re.sub("v", "w", text)  # precedence issue

    '''
    anti-rule 3:
    '''
    text = re.sub(r"i([bdfghžklmnňprsştwýzjçɣq])", r"i\1", text)

    return text

# testing turkmen scripts

def test_turkmen(text):
    input_text = text.lower().split()
    output_text = ipa_to_turkmen(turkmen_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# tatar scripts

def tatar_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # three-sound convenience vowels:
    text = re.sub("[Юю]", "ǔ", text)

    # two-sound convenience consonants:
    text = re.sub("[Цц]", "š", text)
    text = re.sub("[Чч]", "ʆ", text)
    text = re.sub("[Җҗ]", "Ǯ", text)

    # two-sound convenience vowels:
    text = re.sub("[Яя]", "ǎ", text)
    text = re.sub("[Ее]", "ě", text)
    text = re.sub("[Ёё]", "ǒ", text)
    text = re.sub("[Ии]", "ǐ", text)
    text = re.sub("[Уу]", "u", text)

    # single-sound consonants:
    text = re.sub("[Бб]", "b", text)
    text = re.sub("[Вв]", "v", text)
    text = re.sub("[Гг]", "g", text)
    text = re.sub("[Дд]", "d", text)
    text = re.sub("[Жж]", "ʒ", text)
    text = re.sub("[Зз]", "z", text)
    text = re.sub("[Йй]", "j", text)
    text = re.sub("[Кк]", "k", text)
    text = re.sub("[Лл]", "l", text)
    text = re.sub("[Мм]", "m", text)
    text = re.sub("[Нн]", "n", text)
    text = re.sub("[Ңң]", "ŋ", text)
    text = re.sub("[Пп]", "p", text)
    text = re.sub("[Рр]", "r", text)
    text = re.sub("[Сс]", "s", text)
    text = re.sub("[Тт]", "t", text)
    text = re.sub("[Фф]", "f", text)
    text = re.sub("[Хх]", "x", text)
    text = re.sub("[Һһ]", "h", text)
    text = re.sub("[Шш]", "ʃ", text)
    text = re.sub("[Щщ]", "ɕ", text)
    text = re.sub("[Ъъ]", "ʔ", text)
    text = re.sub("[Ьь]", "ʲ", text)

    # single-sound vowels:
    text = re.sub("[Аа]", "ɑ", text)
    text = re.sub("[Әә]", "æ", text)
    text = re.sub("[Оо]", "ɔ", text)
    text = re.sub("[Өө]", "ɵ", text)
    text = re.sub("[Үү]", "ʏ", text)
    text = re.sub("[Ыы]", "ɤ", text)
    text = re.sub("[Ээ]", "e", text)

    # rules 1-4 are similar to those for Kazakh:

    '''
    rule 1:
    if [ě], [ɵ], [ʏ], are followed by [l] and [l] is NOT followed by [ě], [ɵ], [ʏ], or [ʲ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æɵʏě])(l)([^æɵʏěʲ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letters "о" and "ө", [ɔ] and [ɵ] at the beginning of a word are followed by [w] (e.g., осы [wɔsɤ] not [ɔsɤ], өзі [wɵzɪ] not [ɵzɪ]).
    '''
    text = re.sub(r"\b([ɔɵ])", r"w\1", text)

    '''
    rule 3
    if the letter "у" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆǮʲ])", r"w\1", text)

    '''
    rule 4:
    if the letter "и" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆǮʲ])", r"i\1", text)

    # rules 5-6 are specific to Tatar:

    '''
    rule 5:
    а, о, у, ы, ъ + к + а, о, у, ы, ъ
    '''
    text = re.sub(r"k([ɑɔwɤʔ])", r"q\1", text)
    text = re.sub(r"([ɑɔwɤʔ])k", r"\1q", text)

    '''
    rule 6:
    а, о, у, ы, ъ + г + а, о, у, ы, ъ
    '''
    text = re.sub(r"g([ɑɔwɤʔ])", r"ɣ\1", text)
    text = re.sub(r"([ɑɔwɤʔ])g", r"\1ɣ", text)

    return text

def ipa_to_tatar(text):
    # three-sound convenience vowels:
    text = re.sub("ǔ", "ю", text)

    # two-sound convenience consonants:
    text = re.sub("š", "ц", text)
    text = re.sub("ʆ", "ч", text)
    text = re.sub("Ǯ", "җ", text)

    # two-sound convenience vowels:
    text = re.sub("ǎ", "я", text)
    text = re.sub("ě", "е", text)
    text = re.sub("ǒ", "ё", text)
    text = re.sub("ǐ", "и", text)
    text = re.sub("u", "у", text)

    # single-sound consonants:
    text = re.sub("b", "б", text)
    text = re.sub("v", "в", text)
    text = re.sub("g", "г", text)
    text = re.sub("ɣ", "г", text)
    text = re.sub("d", "д", text)
    text = re.sub("ʒ", "ж", text)
    text = re.sub("z", "з", text)
    text = re.sub("j", "й", text)
    text = re.sub("k", "к", text)
    text = re.sub("l", "л", text)
    text = re.sub("m", "м", text)
    text = re.sub("n", "н", text)
    text = re.sub("ŋ", "ң", text)
    text = re.sub("p", "п", text)
    text = re.sub("q", "к", text)
    text = re.sub("r", "р", text)
    text = re.sub("s", "с", text)
    text = re.sub("t", "т", text)
    text = re.sub("f", "ф", text)
    text = re.sub("x", "х", text)
    text = re.sub("h", "һ", text)
    text = re.sub("ʃ", "ш", text)
    text = re.sub("ɕ", "щ", text)
    text = re.sub("ʔ", "ъ", text)
    text = re.sub("ʲ", "ь", text)

    # single-sound vowels:
    text = re.sub("ɑ", "а", text)
    text = re.sub("æ", "ә", text)
    text = re.sub("ɔ", "о", text)
    text = re.sub("ɵ", "ө", text)
    text = re.sub("ʏ", "ү", text)
    text = re.sub("ɤ", "ы", text)
    text = re.sub("e", "э", text)

    # anti-rules 1-4 are similar to those for Kazakh:

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([әөүе])(ł)([^әөүеʲ])", r"\1л\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"\bw([оө])", r"\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"w([бвгдзйклмнңпрстфхһцчшщъьчцжҗqɣ])", r"у\1", text)

    '''
    anti-rule 4:
    '''
    text = re.sub(r"i([бвгдзйклмнңпрстфхһцчшщъьчцжҗqɣ])", r"и\1", text)

    return text

# testing tatar scripts

def test_tatar(text):
    input_text = text.lower().split()
    output_text = ipa_to_tatar(tatar_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# bashkir scripts

def bashkir_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we shall later convert them to conventional symbols.

    # three-sound convenience vowels:
    text = re.sub("[Юю]", "ǔ", text)

    # two-sound convenience consonants:
    text = re.sub("[Цц]", "š", text)
    text = re.sub("[Чч]", "ʆ", text)

    # two-sound convenience vowels:
    text = re.sub("[Яя]", "ǎ", text)
    text = re.sub("[Ее]", "ě", text)
    text = re.sub("[Ёё]", "ǒ", text)
    text = re.sub("[Ии]", "ǐ", text)
    text = re.sub("[Уу]", "u", text)

    # single-sound consonants:
    text = re.sub("[Бб]", "b", text)
    text = re.sub("[Вв]", "v", text)
    text = re.sub("[Гг]", "g", text)
    text = re.sub("[Ғғ]", "ɣ", text)
    text = re.sub("[Дд]", "d", text)
    text = re.sub("[Ҙҙ]", "z", text)
    text = re.sub("[Жж]", "ʒ", text)
    text = re.sub("[Зз]", "z", text)
    text = re.sub("[Йй]", "j", text)
    text = re.sub("[Кк]", "k", text)
    text = re.sub("[Ҡҡ]", "q", text)
    text = re.sub("[Лл]", "l", text)
    text = re.sub("[Мм]", "m", text)
    text = re.sub("[Нн]", "n", text)
    text = re.sub("[Ңң]", "ŋ", text)
    text = re.sub("[Пп]", "p", text)
    text = re.sub("[Рр]", "r", text)
    text = re.sub("[Сс]", "s", text)
    text = re.sub("[Ҫҫ]", "s", text)
    text = re.sub("[Тт]", "t", text)
    text = re.sub("[Хх]", "x", text)
    text = re.sub("[Фф]", "f", text)
    text = re.sub("[Һһ]", "h", text)
    text = re.sub("[Шш]", "ʃ", text)
    text = re.sub("[Щщ]", "ɕ", text)
    text = re.sub("[Ъъ]", "ʔ", text)
    text = re.sub("[Ьь]", "ʲ", text)

    # single-sound vowels:
    text = re.sub("[Аа]", "ɑ", text)
    text = re.sub("[Әә]", "æ", text)
    text = re.sub("[Оо]", "ɔ", text)
    text = re.sub("[Өө]", "ɵ", text)
    text = re.sub("[Үү]", "ʏ", text)
    text = re.sub("[Ыы]", "ɤ", text)
    text = re.sub("[Ээ]", "e", text)

    # rules 1-4 are similar to those for Kazakh:

    '''
    rule 1:
    if [ě], [ɵ], [ʏ], are followed by [l] and [l] is NOT followed by [ě], [ɵ], [ʏ], or [ʲ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æɵʏě])(l)([^æɵʏěʲ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letters "о" and "ө", [ɔ] and [ɵ] at the beginning of a word are followed by [w] (e.g., осы [wɔsɤ] not [ɔsɤ], өзі [wɵzɪ] not [ɵzɪ]).
    '''
    text = re.sub(r"\b([ɔɵ])", r"w\1", text)

    '''
    rule 3
    if the letter "у" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆʲ])", r"w\1", text)

    '''
    rule 4:
    if the letter "и" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdʒzjkqlłmnŋprstfxhʃɕʔšʆʲ])", r"i\1", text)

    return text

def ipa_to_bashkir(text):
    # three-sound convenience vowels:
    text = re.sub("ǔ", "ю", text)

    # two-sound convenience consonants:
    text = re.sub("š", "ц", text)
    text = re.sub("ʆ", "ч", text)

    # two-sound convenience vowels:
    text = re.sub("ě", "е", text)
    text = re.sub("ǒ", "ё", text)
    text = re.sub("ǐ", "и", text)
    text = re.sub("u", "у", text)
    text = re.sub("ǎ", "я", text)

    # single-sound consonants:
    text = re.sub("b", "б", text)
    text = re.sub("v", "в", text)
    text = re.sub("g", "г", text)
    text = re.sub("ɣ", "ғ", text)
    text = re.sub("d", "д", text)
    text = re.sub("z", "з", text)
    text = re.sub("ʒ", "ж", text)
    text = re.sub("j", "й", text)
    text = re.sub("k", "к", text)
    text = re.sub("q", "ҡ", text)
    text = re.sub("l", "л", text)
    text = re.sub("m", "м", text)
    text = re.sub("n", "н", text)
    text = re.sub("ŋ", "ң", text)
    text = re.sub("p", "п", text)
    text = re.sub("r", "р", text)
    text = re.sub("s", "с", text)
    text = re.sub("t", "т", text)
    text = re.sub("f", "ф", text)
    text = re.sub("x", "х", text)
    text = re.sub("h", "һ", text)
    text = re.sub("ʃ", "ш", text)
    text = re.sub("ɕ", "щ", text)
    text = re.sub("ʔ", "ъ", text)
    text = re.sub("ʲ", "ь", text)

    # single-sound vowels:
    text = re.sub("ɑ", "а", text)
    text = re.sub("æ", "ә", text)
    text = re.sub("ɔ", "о", text)
    text = re.sub("ɵ", "ө", text)
    text = re.sub("ʏ", "ү", text)
    text = re.sub("ɤ", "ы", text)
    text = re.sub("e", "э", text)

    # anti-rules 1-4 are similar to those for Kazakh:

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([әөүе])(ł)([^әөүеʲ])", r"\1л\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"\bw([оө])", r"\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"w([бвгғдзйкҡлмнңпрстфхһцчшщъьчцж])", r"у\1", text)

    '''
    anti-rule 4:


    '''
    text = re.sub(r"i([бвгғдзйкҡлмнңпрстфхһцчшщъьчцж])", r"и\1", text)

    return text

# testing bashkir scripts

def test_bashkir(text):
    input_text = text.lower().split()
    output_text = ipa_to_bashkir(bashkir_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# sakha scripts

def sakha_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # three-sound convenience vowels:
    text = re.sub("[Юю]", "ǔ", text)

    # two-sound convenience consonants:
    text = re.sub("[Цц]", "š", text)
    text = re.sub("[Чч]", "ʆ", text)
    text = re.sub("ДЬ", "Ǯ", text)
    text = re.sub("дь", "Ǯ", text)
    text = re.sub("Дь", "Ǯ", text)
    text = re.sub("дЬ", "Ǯ", text)
    text = re.sub("НЬ", "ɲ", text)
    text = re.sub("нь", "ɲ", text)
    text = re.sub("Нь", "ɲ", text)
    text = re.sub("нЬ", "ɲ", text)

    # two-sound convenience vowels:
    text = re.sub("[Яя]", "ǎ", text)
    text = re.sub("[Ее]", "ě", text)
    text = re.sub("[Ёё]", "ǒ", text)
    text = re.sub("[Ии]", "ǐ", text)
    text = re.sub("[Уу]", "u", text)

    # single-sound consonants:
    text = re.sub("[Бб]", "b", text)
    text = re.sub("[Вв]", "v", text)
    text = re.sub("[Гг]", "g", text)
    text = re.sub("[Ҕҕ]", "ɣ", text)
    text = re.sub("[Дд]", "d", text)
    text = re.sub("[Жж]", "ʒ", text)
    text = re.sub("[Зз]", "z", text)
    text = re.sub("[Йй]", "j", text)
    text = re.sub("[Кк]", "k", text)
    text = re.sub("[Лл]", "l", text)
    text = re.sub("[Мм]", "m", text)
    text = re.sub("[Нн]", "n", text)
    text = re.sub("[Ҥҥ]", "ŋ", text)
    text = re.sub("[Пп]", "p", text)
    text = re.sub("[Рр]", "r", text)
    text = re.sub("[Сс]", "s", text)
    text = re.sub("[Тт]", "t", text)
    text = re.sub("[Хх]", "x", text)
    text = re.sub("[Фф]", "f", text)
    text = re.sub("[Һһ]", "h", text)
    text = re.sub("[Шш]", "ʃ", text)
    text = re.sub("[Щщ]", "ɕ", text)
    text = re.sub("[Ъъ]", "ʔ", text)
    text = re.sub("[Ьь]", "ʲ", text)

    # single-sound vowels:
    text = re.sub("[Аа]", "ɑ", text)
    text = re.sub("[Әә]", "æ", text)
    text = re.sub("[Оо]", "ɔ", text)
    text = re.sub("[Өө]", "ɵ", text)
    text = re.sub("[Үү]", "ʏ", text)
    text = re.sub("[Ыы]", "ɤ", text)
    text = re.sub("[Ээ]", "e", text)

    # rules 1-4 are similar to those for Kazakh:

    '''
    rule 1:
    if [ě], [ɵ], [ʏ], are followed by [l] and [l] is NOT followed by [ě], [ɵ], [ʏ], or [ʲ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æɵʏě])(l)([^æɵʏěʲ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letters "о" and "ө", [ɔ] and [ɵ] at the beginning of a word are followed by [w] (e.g., осы [wɔsɤ] not [ɔsɤ], өзі [wɵzɪ] not [ɵzɪ]).
    '''
    text = re.sub(r"\b([ɔɵ])", r"w\1", text)

    '''
    rule 3
    if the letter "у" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdʒzjklłmnŋɲprstfxhʃɕʔšʆǮʲ])", r"w\1", text)

    '''
    rule 4:
    if the letter "и" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdʒzjklłmnŋɲprstfxhʃɕʔšʆǮʲ])", r"i\1", text)

    return text

def ipa_to_sakha(text):
    # three-sound convenience vowels:
    text = re.sub("ǔ", "ю", text)

    # two-sound convenience consonants:
    text = re.sub("š", "ц", text)
    text = re.sub("ʆ", "ч", text)
    text = re.sub("Ǯ", "дь", text)
    text = re.sub("ɲ", "нь", text)

    # two-sound convenience vowels:
    text = re.sub("ě", "е", text)
    text = re.sub("ǒ", "ё", text)
    text = re.sub("ǐ", "и", text)
    text = re.sub("u", "у", text)
    text = re.sub("ǎ", "я", text)

    # single-sound consonants:
    text = re.sub("b", "б", text)
    text = re.sub("v", "в", text)
    text = re.sub("g", "г", text)
    text = re.sub("ɣ", "ҕ", text)
    text = re.sub("d", "д", text)
    text = re.sub("z", "з", text)
    text = re.sub("ʒ", "ж", text)
    text = re.sub("j", "й", text)
    text = re.sub("k", "к", text)
    text = re.sub("l", "л", text)
    text = re.sub("m", "м", text)
    text = re.sub("n", "н", text)
    text = re.sub("ŋ", "ҥ", text)
    text = re.sub("p", "п", text)
    text = re.sub("r", "р", text)
    text = re.sub("s", "с", text)
    text = re.sub("t", "т", text)
    text = re.sub("f", "ф", text)
    text = re.sub("x", "х", text)
    text = re.sub("h", "һ", text)
    text = re.sub("ʃ", "ш", text)
    text = re.sub("ɕ", "щ", text)
    text = re.sub("ʔ", "ъ", text)
    text = re.sub("ʲ", "ь", text)

    # single-sound vowels:
    text = re.sub("ɑ", "а", text)
    text = re.sub("æ", "ә", text)
    text = re.sub("ɔ", "о", text)
    text = re.sub("ɵ", "ө", text)
    text = re.sub("ʏ", "ү", text)
    text = re.sub("ɤ", "ы", text)
    text = re.sub("e", "э", text)

    # anti-rules 1-4 are similar to those for Kazakh:

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([әөүе])(ł)([^әөүеʲ])", r"\1л\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"\bw([оө])", r"\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"w(дь)", r"у\1", text)
    text = re.sub(r"w(нь)", r"у\1", text)
    text = re.sub(r"w([бвгҕдзйклмнҥпрстфхһцчшщъьчцж])", r"у\1", text)

    '''
    anti-rule 4:
    '''
    text = re.sub(r"i(дь)", r"и\1", text)
    text = re.sub(r"i(нь)", r"и\1", text)
    text = re.sub(r"i([бвгҕдзйклмнҥпрстфхһцчшщъьчцж])", r"и\1", text)

    return text

# testing sakha scripts

# testing bashkir scripts

def test_sakha(text):
    input_text = text.lower().split()
    output_text = ipa_to_sakha(sakha_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

# uyghur scripts

def uyghur_to_ipa(text):
    # we shall begin with sound combinations:
    # the longer a combination, the upper it is on the list
    # single sounds should go to the bottom, with consonants taking precedence over vowels.
    # consonants are less likely to change than vowels.

    # for convenience, we shall use single symbols to denote multiple-sound combinations.
    # we can later convert them to conventional symbols.

    # two-sound convenience consonants:
    text = re.sub("[Jj]", "Ǯ", text)
    text = re.sub("Ch", "ʆ", text)
    text = re.sub("ch", "ʆ", text)

    # two-sound convenience vowels:
    text = re.sub("[Ii]", "ǐ", text)
    text = re.sub("[Uu]", "u", text)

    # single-sound consonants:
    text = re.sub("[Bb]", "b", text)
    text = re.sub("[Dd]", "d", text)
    text = re.sub("[Ff]", "f", text)
    text = re.sub("Gh", "ɣ", text)
    text = re.sub("gh", "ɣ", text)
    text = re.sub("[Gg]", "g", text)
    text = re.sub("[Hh]", "h", text)
    text = re.sub("[Kk]", "k", text)
    text = re.sub("[Ll]", "l", text)
    text = re.sub("[Mm]", "m", text)
    text = re.sub("[Nn]", "n", text)
    text = re.sub("Ng", "ŋ", text)
    text = re.sub("ng", "ŋ", text)
    text = re.sub("[Pp]", "p", text)
    text = re.sub("[Qq]", "q", text)
    text = re.sub("[Rr]", "r", text)
    text = re.sub("[Ss]", "s", text)
    text = re.sub("Sh", "ʃ", text)
    text = re.sub("sh", "ʃ", text)
    text = re.sub("[Tt]", "t", text)
    text = re.sub("[Ww]", "v", text)
    text = re.sub("[Xx]", "x", text)
    text = re.sub("[Yy]", "j", text)
    text = re.sub("[Zz]", "z", text)
    text = re.sub("Zh", "ʒ", text)
    text = re.sub("zh", "ʒ", text)

    # single-sound vowels:
    text = re.sub("[Aa]", "ɑ", text)
    text = re.sub("[Ee]", "æ", text)
    text = re.sub("[ËÉëé]", "e", text)
    text = re.sub("[Oo]", "ɔ", text)
    text = re.sub("[Öö]", "ɵ", text)
    text = re.sub("[Üü]", "ʏ", text)

    # hard sign
    text = re.sub("'", "ʔ", text)

    '''
    rule 1:
    if [æ], [e], [ɵ], [ǐ] are followed by [l] and [l] is NOT followed by [æ], [e], [ɵ], [ǐ],
    use [ł] instead of [l].
    '''
    text = re.sub(r"([æɵǐeʏ])(l)([^æɵǐeʏ])", r"\1ł\3", text)

    '''
    rule 2:
    if the letter "u" [u] is followed by consonants, use [w] instead of [u].
    '''
    text = re.sub(r"u([bvgɣdzjkqlłmnŋprstfxhʃʆǮʒ])", r"w\1", text)

    '''
    rule 3:
    if the letter "i" [ǐ] is followed by consonants, use [i] instead of [ǐ].
    '''
    text = re.sub(r"ǐ([bvgɣdzjkqlłmnŋprstfxhʃʆǮʒ])", r"i\1", text)

    return text

def ipa_to_uyghur(text):
    # two-sound convenience consonants:
    text = re.sub("j", "y", text)  # exception! precedence issue
    text = re.sub("Ǯ", "j", text)
    text = re.sub("ʆ", "ch", text)
    text = re.sub("ʒ", "zh", text)

    # two-sound convenience vowels:
    text = re.sub("ǐ", "i", text)
    text = re.sub("u", "u", text)

    # single-sound convenience consonants:
    text = re.sub("b", "b", text)
    text = re.sub("d", "d", text)
    text = re.sub("f", "f", text)
    text = re.sub("g", "g", text)
    text = re.sub("ɣ", "gh", text)
    text = re.sub("h", "h", text)
    text = re.sub("k", "k", text)
    text = re.sub("l", "l", text)
    text = re.sub("m", "m", text)
    text = re.sub("n", "n", text)
    text = re.sub("ŋ", "ng", text)
    text = re.sub("p", "p", text)
    text = re.sub("q", "q", text)
    text = re.sub("r", "r", text)
    text = re.sub("s", "s", text)
    text = re.sub("ʃ", "sh", text)
    text = re.sub("t", "t", text)
    text = re.sub("v", "w", text)
    text = re.sub("x", "x", text)
    text = re.sub("z", "z", text)

    # single-sound convenience vowels:
    text = re.sub("ɑ", "a", text)
    text = re.sub("e", "ë", text) # precedence
    text = re.sub("æ", "e", text)
    text = re.sub("ɵ", "ö", text)
    text = re.sub("ɔ", "o", text)
    text = re.sub("ʏ", "ü", text)

    # hard sign
    text = re.sub("ʔ", "'", text)

    '''
    anti-rule 1:
    '''
    text = re.sub(r"([eëiöü])(ł)([^eëiöü])", r"\1l\3", text)

    '''
    anti-rule 2:
    '''
    text = re.sub(r"w([bcvgdjzklmnpqrstfhyx])", r"u\1", text)

    '''
    anti-rule 3:
    '''
    text = re.sub(r"i([bcvgdjzklmnpqrstfhyx])", r"i\1", text)

    return text

# testing uyghur scripts

def test_uyghur(text):
    input_text = text.lower().split()
    output_text = ipa_to_uyghur(uyghur_to_ipa(text)).split()

    input_difference = []
    output_difference = []
    for item in input_text:
        if item not in output_text:
            input_difference.append(item)
    for item in output_text:
        if item not in input_text:
            output_difference.append(item)

    if input_text == output_text:
        print("input text and output text -- identical")
    else:
        print("input text and output text -- different")
        print("input:", input_difference)
        print("output:", output_difference)

In [ ]:
import ipa_convert

dispatcher = {'kazakh' : ipa_convert.kazakh_to_ipa, 
              'turkish' : ipa_convert.turkish_to_ipa,
              'azerbaijani': ipa_convert.azerbaijani_to_ipa,
              'kyrgyz' : ipa_convert.kyrgyz_to_ipa,              
              'uzbek' : ipa_convert.uzbek_to_ipa,
              'turkmen' : ipa_convert.turkmen_to_ipa,
              'tatar' : ipa_convert.tatar_to_ipa,
              'bashkir' : ipa_convert.bashkir_to_ipa,
              'sakha' : ipa_convert.sakha_to_ipa,
              'uyghur' : ipa_convert.uyghur_to_ipa,              
              'tts_sent' : ipa_convert.ipa_to_kazakh}

def call_func(x, func):
    try:
        return dispatcher[func](x)
    except:
        return "Invalid function"

def normalization(x, lang="kazakh"):
    ipa_text = call_func(x, lang)
    kz_text = call_func(ipa_text, 'tts_sent')
    return kz_text

In [ ]:
from parallel_wavegan.utils import load_model
from espnet2.bin.tts_inference import Text2Speech
from scipy.io.wavfile import write
# from utils import normalization
import torch

fs = 22050
vocoder_checkpoint="parallelwavegan_male2_checkpoint/checkpoint-400000steps.pkl" ### specify vocoder path
vocoder = load_model(vocoder_checkpoint).to("cuda").eval()
vocoder.remove_weight_norm()

## specify path to the main model(transformer/tacotron2/fastspeech) and its config file
config_file = "exp/tts_train_raw_char/config.yaml"
model_path = "exp/tts_train_raw_char/train.loss.ave_5best.pth"

text2speech = Text2Speech(
    config_file,
    model_path,
    device="cuda", ## if cuda not available use cpu
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=True,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
text2speech.spc2wav = None  # Disable griffin-lim

# text = """Iqlimshunos Erkin Abdulahatov agar yo‘l qurilishida tartiblar ketma-ketligida ishlar to‘g‘ri bajarilmasa, barpo etilgan infratuzilma birinchi kuchli tabiat hodisasiga dosh bera olmasligini ta’kidlaydi."""

# text = """Ақын саяси қызметі үшін 1870 жылдары Санкт-Петербургтен Семейге айдалып келген Михаэлиспен, 80-жылдарда орыс демократтары Н. И. Долгополов, А. А.Леонтьевпен танысады."""
# text = """татарларның милли теле, Татарстанның дәүләт теле, Русиядә таралу буенча икенче тел"""
### available options are: kazakh, turkish, azerbaijani, kyrgyz, uzbek, turkmen, tatar, bashkir, sakha, uyghur
text = 'бірінші сабақ башқұрт тілінің үҙенсәлектәре'
lang = "kazakh"
 
text = normalization(text, lang)
with torch.no_grad():
    c_mel = text2speech(text)['feat_gen']
    wav = vocoder.inference(c_mel)
write("result.wav", fs, wav.view(-1).cpu().numpy())

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.display(ipd.Audio(wav.view(-1).cpu().numpy(), rate=fs))